In [4]:
import pandas as pd
import numpy as np
import os
import ast

from datetime import datetime


CARGA DE DATA

In [14]:
url='https://drive.google.com/file/d/1yqB8ahhhdiONfZLu8-xuU9qsWOgSf5Oo/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_labs = pd.read_csv(url)

C:\Users\pecor\AppData\Local\Temp\ipykernel_20744\3032137973.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_labs = pd.read_csv(url)


In [16]:
df_labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [17]:
df_labs.shape

(45466, 24)

                            DESARROLLO Y TRANSFORMACION DE DATOS

01.ELIMINAR COLUMNAS

In [27]:
df_labs.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage', 'tagline'], axis=1, inplace = True)

02.VALORES NULOS REVENUE Y BUDGET

In [28]:
df_labs['budget'] = pd.to_numeric(df_labs['budget'], errors='coerce').fillna(0)

In [29]:
df_labs['revenue'] = df_labs['revenue'].fillna(0)

03.Crear Columna Return

In [30]:
df_labs['return'] = df_labs.apply(lambda x: x['revenue']/x['budget'] if x['budget'] != 0 else 0, axis=1)

In [31]:
df_labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  float64
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   45463 non-null  object 
 8   production_countries   45463 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45466 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  title                  45460 non-null  object 
 15  vo

04. DESANIDAR 

04.1 String a Diccionario

In [32]:
def str_to_dict(value):
    """Convert string a diccionario

    Args:
        value (str): String valu se convierte en diccionario

    Returns:
        None : Return None if NaN value is present
    """
    if pd.isna(value):
        return None
    else:
        return ast.literal_eval(value)

04.2 Desanidar Columnas

In [33]:
df_labs['belongs_to_collection'] = df_labs['belongs_to_collection'].apply(str_to_dict)

In [34]:
df_nested = pd.json_normalize(df_labs['belongs_to_collection']).fillna('')
df_nested.head()

,id,name,poster_path,backdrop_path
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,,,,
2,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,,,,
4,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


In [35]:
columns = df_nested.columns
new_columns_names = {column : f'belongs_to_collections_{column}' for column in columns}
df_nested.rename(columns=new_columns_names, inplace=True)
df_nested.head()

,belongs_to_collections_id,belongs_to_collections_name,belongs_to_collections_poster_path,belongs_to_collections_backdrop_path
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,,,,
2,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,,,,
4,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg


In [36]:
df_nested.shape

(45466, 4)

In [37]:
columns = ['genres', 'production_companies', 'production_countries', 'spoken_languages']

for column in columns:
    df_labs[column] = df_labs[column].apply(str_to_dict)
    df_nested = pd.json_normalize(df_labs[column]).fillna('')
    nested_columns = df_nested.columns
    df_nested_1 = pd.DataFrame()
    for nested_column in nested_columns:
        df_aux = pd.json_normalize(df_nested[nested_column]).fillna('')
        if 'production_countries' in column:
            df_aux.rename(columns={'iso_3166_1': 'id'}, inplace=True)
        if 'spoken_languages' in column:
            df_aux.rename(columns={'iso_639_1': 'id'}, inplace=True)
        df_aux.drop(columns='id', axis=1, inplace=True)
        new_columns_names = {col : f'{column}_{nested_column}_{col}' for col in df_aux.columns}
        df_aux.rename(columns=new_columns_names, inplace=True)
        df_nested_1 = pd.concat([df_nested_1, df_aux], axis=1)

    # Convertir columns to a list of columns
    column_name = f'{column}_name'
    column_name_list = df_nested_1.columns.to_list()
    df_nested_1[column_name] = df_nested_1.values.tolist()
    df_nested_1.drop(columns=column_name_list, axis=1, inplace=True)
    df_labs = pd.concat([df_labs, df_nested_1], axis=1)
    df_labs.drop(columns=column, inplace=True)

In [38]:
df_labs.shape

(45466, 18)

In [39]:
df_labs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   belongs_to_collection      4494 non-null   object 
 1   budget                     45466 non-null  float64
 2   id                         45466 non-null  object 
 3   original_language          45455 non-null  object 
 4   overview                   44512 non-null  object 
 5   popularity                 45461 non-null  object 
 6   release_date               45379 non-null  object 
 7   revenue                    45466 non-null  float64
 8   runtime                    45203 non-null  float64
 9   status                     45379 non-null  object 
 10  title                      45460 non-null  object 
 11  vote_average               45460 non-null  float64
 12  vote_count                 45460 non-null  float64
 13  return                     45466 non-null  flo

In [40]:
df_labs['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45466, dtype: object

05. VALORES NULOS RELEASE_DATE

In [41]:
df_labs.dropna(subset=['release_date'], inplace=True)
df_labs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45379 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   belongs_to_collection      4491 non-null   object 
 1   budget                     45379 non-null  float64
 2   id                         45379 non-null  object 
 3   original_language          45368 non-null  object 
 4   overview                   44438 non-null  object 
 5   popularity                 45377 non-null  object 
 6   release_date               45379 non-null  object 
 7   revenue                    45379 non-null  float64
 8   runtime                    45130 non-null  float64
 9   status                     45296 non-null  object 
 10  title                      45376 non-null  object 
 11  vote_average               45376 non-null  float64
 12  vote_count                 45376 non-null  float64
 13  return                     45379 non-null  flo

06. FORMATO FECHA

06.1 CREAR COLUMNA RELEASE YEAR

In [42]:
regex_filter = '^\d{4}\-(0?[1-9]|1[012])\-(0?[1-9]|[12][0-9]|3[01])$'
filter = df_labs['release_date'].str.contains(regex_filter)
df_labs = df_labs[filter]
df_labs['release_year'] = df_labs['release_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date().year)

C:\Users\pecor\AppData\Local\Temp\ipykernel_18308\312493890.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filter = df_labs['release_date'].str.contains(regex_filter)


06.2 CONVERTIR EL RELEASE DATE A FORMATO FECHA

In [43]:
df_labs['release_date'] = pd.to_datetime(df_labs['release_date'], format='%Y-%m-%d')

In [44]:
print(type(df_labs))

<class 'pandas.core.frame.DataFrame'>


MODULO API

In [45]:
#from fastapi import FastAPI
#from fastapi.responses import JSONResponse
#import pandas as pd

#app = FastAPI()

#@app.get("/dataframe")
#def get_dataframe():
#   data = df_labs
#   return JSONResponse(content=data.to_json(orient='records'), media_type='application/json')

#import nest_asyncio
#nest_asyncio.apply()
#import uvicorn

#if __name__ == '__main__':
#    uvicorn.run(app, host='0.0.0.0', port=8000)

In [48]:
df_labs['release_date'] = pd.to_datetime(df_labs['release_date'])

In [51]:
fecha_filtrada = pd.to_datetime('2010-06-10')  # Fecha específica a filtrar

df_filtrado = df_labs[df_labs['release_date'] == fecha_filtrada]

In [53]:
filas = df_filtrado.loc[:]

In [55]:
print(filas)

      belongs_to_collection       budget      id original_language  \
15344                  None  110000000.0   34544                en   
15496                  None   40000000.0   38575                en   
18185                  None   50000000.0   38407                en   
18624                  None          0.0   51875                en   
26024                  None   10000000.0   51250                en   
40458                  None          0.0  352812                en   
41123                  None          0.0  113649                ko   

                                                overview popularity  \
15344  A group of Iraq War veterans goes on the run f...  16.747793   
15496  12-year-old Dre Parker could have been the mos...  11.690781   
18185  An American man returns to a corrupt, Japanese...   7.370332   
18624  A mild-mannered secretary discovers that she h...   0.774843   
26024        A drama set in the world of weapon dealing.   2.834545   
40458  Amidst